Do create file follow ("video<>.mp4")

In [3]:
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO
import tkinter as tk
from PIL import Image, ImageTk

# โหลดข้อมูลจากไฟล์ map_.csv
df = pd.read_csv('map_.csv')
raw = df.to_numpy()
size = (raw[0][0], raw[0][1])

# ฟังก์ชันสำหรับบันทึกข้อมูลแผนที่เป็น CSV
def save_map(cfg_np: np.ndarray, org_df: pd.DataFrame, asMdf=False, asNew=False, location: str="./save/map.csv") -> int:
    row_count, col_count = cfg_np.shape[0], cfg_np.shape[1] - 2
    cfg_np[0, 0] = row_count
    cfg_np[0, 1] = col_count
    df_new = pd.DataFrame(cfg_np, columns=org_df.columns)
    if asNew:
        df_new.to_csv(location, index=False)
    elif asMdf:
        df_new.to_csv('map_modified.csv', index=False)
    else:
        df_new.to_csv('map.csv', index=False)
    return 0

# โหลดโมเดล YOLOv8
model = YOLO("yolov8n.pt")

# ฟังก์ชันตรวจจับบุคคล
def detect_people(frame):
    results = model(frame)
    detections = results[0].boxes.xyxy.cpu().numpy()
    people_count = len(detections)
    return people_count, detections

# ฟังก์ชันประมวลผลวิดีโอ
def process_video(cap, zone_row, zone_column, map_df, video_label, zoom_out=False):
    ret, frame = cap.read()
    if not ret:
        return 0, False

    # ตรวจจับบุคคลในเฟรม
    people_count, detections = detect_people(frame)

    # แสดงกรอบรอบ ๆ บุคคลที่ตรวจพบในวิดีโอ
    color = (0, 255, 0)
    for (x1, y1, x2, y2) in detections:
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        label = "Person"
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # ซูมออกสำหรับวิดีโอ 4, 5, 6 (ซูมออก 4 เท่า)
    if zoom_out:
        frame = cv2.resize(frame, None, fx=0.25, fy=0.25)

    # ย่อขนาดเฟรมเพื่อให้แสดงใน GUI
    frame = cv2.resize(frame, (320, 240))

    # แปลงเฟรมจาก OpenCV เป็น ImageTk
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    img_tk = ImageTk.PhotoImage(image=img)
    video_label.config(image=img_tk)
    video_label.image = img_tk

    # จำกัดจำนวนคนให้อยู่ระหว่าง 20 ถึง 40
    people_count = max(20, min(40, people_count))
    map_df.iloc[zone_row, zone_column + 2] = people_count

    return people_count, True

# ตั้งค่า Tkinter GUI
root = tk.Tk()
root.title("Video Streams")
root.geometry("1000x800")

# วาง Label สำหรับแสดงผลวิดีโอ
video_labels = [tk.Label(root) for _ in range(3)]
for i, video_label in enumerate(video_labels):
    video_label.grid(row=i//3, column=i%3, padx=5, pady=5)

# กำหนดรายการวิดีโอและตำแหน่งโซน
video_files = ["1video.mp4", "2video.mp4", "3video.mp4", "4video.mp4", "5video.mp4", "6video.mp4"]
zones = [(0, 2), (1, 2), (2, 2), (3, 2), (4, 2), (5, 2)]
caps = [cv2.VideoCapture(video) for video in video_files]
running = [cap.isOpened() for cap in caps]

def update_videos():
    for i, cap in enumerate(caps[:3]):
        if running[i]:
            zone_row, zone_column = zones[i]
            video_label = video_labels[i % 3]
            zoom_out = (i >= 3)  # ซูมออกสำหรับวิดีโอที่ 4, 5, และ 6
            total_count, status = process_video(cap, zone_row, zone_column, df, video_label, zoom_out=zoom_out)
            if not status:
                running[i] = False
    root.after(10, update_videos)

# เริ่มการแสดงวิดีโอ
update_videos()

# รัน GUI
root.mainloop()

# ปิดวิดีโอและบันทึก map.csv
for cap in caps:
    cap.release()
cv2.destroyAllWindows()
save_map(df.to_numpy(), df, asNew=True, location="./save/updated_map.csv")



0: 384x640 37 persons, 2 birds, 59.9ms
Speed: 3.4ms preprocess, 59.9ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 32.2ms
Speed: 1.6ms preprocess, 32.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 4 cars, 1 truck, 3 potted plants, 26.1ms
Speed: 1.0ms preprocess, 26.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 2 birds, 37.0ms
Speed: 1.6ms preprocess, 37.0ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.6ms
Speed: 1.2ms preprocess, 28.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 4 cars, 1 truck, 3 potted plants, 30.9ms
Speed: 1.3ms preprocess, 30.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 34 persons, 2 birds, 29.9ms
Speed: 1.3ms preprocess, 29.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)



Speed: 1.3ms preprocess, 51.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 30.2ms
Speed: 1.4ms preprocess, 30.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 5 cars, 1 truck, 3 potted plants, 29.8ms
Speed: 1.3ms preprocess, 29.8ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 32 persons, 2 birds, 1 dog, 34.5ms
Speed: 1.3ms preprocess, 34.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.4ms
Speed: 0.9ms preprocess, 29.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 4 cars, 1 truck, 3 potted plants, 28.6ms
Speed: 1.0ms preprocess, 28.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 34 persons, 1 bird, 1 dog, 29.5ms
Speed: 1.1ms preprocess, 29.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections),

Speed: 1.1ms preprocess, 29.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.6ms
Speed: 1.4ms preprocess, 29.6ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 4 cars, 1 truck, 2 potted plants, 32.0ms
Speed: 1.4ms preprocess, 32.0ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 36 persons, 2 birds, 29.0ms
Speed: 1.2ms preprocess, 29.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 33.1ms
Speed: 1.1ms preprocess, 33.1ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 4 cars, 1 truck, 2 potted plants, 34.3ms
Speed: 1.8ms preprocess, 34.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 35 persons, 2 birds, 32.5ms
Speed: 1.1ms preprocess, 32.5ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 34.1ms
Speed


0: 384x640 38 persons, 1 bird, 29.2ms
Speed: 1.3ms preprocess, 29.2ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 30.5ms
Speed: 1.1ms preprocess, 30.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 4 cars, 1 truck, 2 potted plants, 31.1ms
Speed: 1.5ms preprocess, 31.1ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 1 bird, 30.1ms
Speed: 1.3ms preprocess, 30.1ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.9ms
Speed: 1.4ms preprocess, 29.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 5 cars, 1 truck, 2 potted plants, 28.3ms
Speed: 1.1ms preprocess, 28.3ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 38 persons, 1 bird, 28.8ms
Speed: 1.3ms preprocess, 28.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 


0: 384x640 (no detections), 30.5ms
Speed: 1.3ms preprocess, 30.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 5 cars, 1 truck, 2 potted plants, 31.0ms
Speed: 1.2ms preprocess, 31.0ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 41 persons, 29.8ms
Speed: 1.4ms preprocess, 29.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 33.7ms
Speed: 1.4ms preprocess, 33.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 5 cars, 1 truck, 2 potted plants, 29.9ms
Speed: 1.3ms preprocess, 29.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 40 persons, 2 birds, 28.5ms
Speed: 1.4ms preprocess, 28.5ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.8ms
Speed: 1.0ms preprocess, 28.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 

...

Speed: 0.9ms preprocess, 26.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 33.8ms
Speed: 1.4ms preprocess, 33.8ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.0ms
Speed: 1.1ms preprocess, 29.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 33.0ms
Speed: 1.5ms preprocess, 33.0ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 34.4ms
Speed: 1.6ms preprocess, 34.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.8ms
Speed: 1.0ms preprocess, 28.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.2ms
Speed: 1.3ms preprocess, 29.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.4ms
Speed: 1.1ms preprocess, 29.4ms inference, 0.3ms postprocess per imag


0: 384x640 (no detections), 28.5ms
Speed: 1.1ms preprocess, 28.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.5ms
Speed: 1.3ms preprocess, 28.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.1ms
Speed: 1.3ms preprocess, 29.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.4ms
Speed: 1.3ms preprocess, 28.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.4ms
Speed: 1.2ms preprocess, 27.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.1ms
Speed: 1.2ms preprocess, 28.1ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.0ms
Speed: 1.1ms preprocess, 29.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.7ms
Speed: 1.1ms preprocess, 26.7ms i


0: 384x640 (no detections), 26.3ms
Speed: 1.2ms preprocess, 26.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.9ms
Speed: 1.1ms preprocess, 26.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.4ms
Speed: 1.1ms preprocess, 27.4ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.2ms
Speed: 1.1ms preprocess, 28.2ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.8ms
Speed: 1.1ms preprocess, 27.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 32.9ms
Speed: 1.5ms preprocess, 32.9ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.8ms
Speed: 1.1ms preprocess, 26.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.4ms
Speed: 1.1ms preprocess, 26.4ms i


0: 384x640 (no detections), 26.3ms
Speed: 1.0ms preprocess, 26.3ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.8ms
Speed: 1.1ms preprocess, 28.8ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 29.0ms
Speed: 1.1ms preprocess, 29.0ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.7ms
Speed: 1.3ms preprocess, 27.7ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.6ms
Speed: 1.1ms preprocess, 28.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 27.7ms
Speed: 1.0ms preprocess, 27.7ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.6ms
Speed: 1.2ms preprocess, 28.6ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 28.7ms
Speed: 1.1ms preprocess, 28.7ms i

KeyboardInterrupt: 